In [1]:
## Iván Andrés Trujillo Abella (last modification: 7/17/2023)
## ivantrujillo1229@gmail.com

In [2]:
import pandas as pd

In [3]:
# The dataframes contain one key that is identification...

In [4]:
store = {}
for x in range(1,10):
  store['var'+str(x)] = x**0.5

In [5]:
## Open several datasets

In [6]:
%%writefile data2010.csv
NIT, var1, var2, var3, year
1, 'A', 'B', 'C', 2010
2, 10, '12', '12', 2010

Writing data2010.csv


In [7]:
%%writefile data2011.csv
NIT, var1, var2, var3, year, indicator
1, 'A', 'B', 'C', 2011
2, 10, '12', '12', 2011
3, tres,'tres',10 , 2011, new

Writing data2011.csv


In [8]:
%%writefile data2012.csv
NIT, var1, var2, var3, year, indicator, cuatro
1, 'A', 'B', 'C', 2012, old
2, 10, '12', '12', 2012, old
3, tres,'tres',10 , 2012, old
4, 'cuatro', 'cuatro', 'cuatro', 2012 , new, yes

Writing data2012.csv


In [9]:
def new_firms(year, lastYear):
  if year < lastYear:
    news = set(dfs['df'+str(year+1)]['NIT'].unique()) - set(dfs['df'+str(year)]['NIT'].unique())
    firms =  dfs['df'+str(year+1)]['NIT'].isin(news)
    return dfs['df'+str(year+1)][firms]

In [10]:
dfs = {}
for year in [2010, 2011, 2012]:
  dfs['df'+str(year)] = pd.read_csv('data'+str(year)+'.csv')

In [11]:
new_firms(2010,2012)

,NIT,var1,var2,var3,year,indicator
2,3,tres,'tres',10,2011,new


In [12]:
full = dfs['df2010'].copy()
for year in [2010,2011,2012]:
  full = full.merge(dfs['df'+str(year)], on='NIT', how='inner', suffixes=(None, 'to_remove'))
  duplicated  = [var for var in full.columns if 'to_remove' in var ]
  full.drop(columns=duplicated, inplace=True)
  full = pd.concat([full,new_firms(year, 2012)])

In [13]:
full

,NIT,var1,var2,var3,year,indicator,cuatro
0,1,'A','B','C',2010,NaN,NaN
1,2,10,'12','12',2010,NaN,NaN
2,3,tres,'tres',10,2011,new,NaN
3,4,'cuatro','cuatro','cuatro',2012,new,yes


In [14]:
dfs['df2010']

,NIT,var1,var2,var3,year
0,1,'A','B','C',2010
1,2,10,'12','12',2010


In [15]:
dfs['df2011']

,NIT,var1,var2,var3,year,indicator
0,1,'A','B','C',2011,NaN
1,2,10,'12','12',2011,NaN
2,3,tres,'tres',10,2011,new


In [16]:
dfs['df2010'].merge(dfs['df2011'], on='NIT', how='inner')

,NIT,var1_x,var2_x,var3_x,year_x,var1_y,var2_y,var3_y,year_y,indicator
0,1,'A','B','C',2010,'A','B','C',2011,NaN
1,2,10,'12','12',2010,10,'12','12',2011,NaN


In [17]:
## all is ok?

In [18]:
%%writefile data2010.csv
NIT, var1, var2, var3, year
1, 'A', 'B', 'C', 2010
2, 10, '12', '12', 2010

Overwriting data2010.csv


In [19]:
%%writefile data2011.csv
NIT, var1, var2, var3, year, indicator
1, 'A', 'B', 'C', 2011
3, tres,'tres',10 , 2011, new

Overwriting data2011.csv


In [20]:
%%writefile data2012.csv
NIT, var1, var2, var3, year, indicator, cuatro
1, 'A', 'B', 'C', 2012, old
3, tres,'tres',10 , 2012, old
4, 'cuatro', 'cuatro', 'cuatro', 2012 , new, yes

Overwriting data2012.csv


In [21]:
dfs = {}
for year in [2010, 2011, 2012]:
  dfs['df'+str(year)] = pd.read_csv('data'+str(year)+'.csv')

In [22]:
full = dfs['df2010'].copy()
for year in [2010,2011,2012]:
  full = full.merge(dfs['df'+str(year)], on='NIT', how='inner', suffixes=(None, 'to_remove'))
  duplicated  = [var for var in full.columns if 'to_remove' in var ]
  full.drop(columns=duplicated, inplace=True)
  full = pd.concat([full,new_firms(year, 2012)])

In [23]:
full # What happend?? where is two??

,NIT,var1,var2,var3,year,indicator,cuatro
0,1,'A','B','C',2010,NaN,NaN
1,3,tres,'tres',10,2011,new,NaN
2,4,'cuatro','cuatro','cuatro',2012,new,yes


In [24]:
def new_firms(current_df, next_df, key):
  firm_first =  set(current_df[key].unique()) -  set(next_df[key].unique())
  firm_second =  set(next_df[key].unique()) -  set(current_df[key].unique())
  return pd.concat([current_df[current_df[key].isin(firm_first)] , next_df[next_df[key].isin(firm_second)]])

In [25]:
full = dfs['df2010'].copy()
firms = new_firms(full, dfs['df2011'], 'NIT') # Drawn new firms in both datasets..
full = full.merge(dfs['df2011'], on='NIT', how='inner', suffixes=(None, 'to_remove'))
duplicated  = [var for var in full.columns if 'to_remove' in var ]
full.drop(columns=duplicated, inplace=True)
full = pd.concat([full, firms])

In [26]:
full

,NIT,var1,var2,var3,year,indicator
0,1,'A','B','C',2010,NaN
1,2,10,'12','12',2010,NaN
1,3,tres,'tres',10,2011,new


In [27]:
### Test loops
full = dfs['df2010'].copy()
for year in [2011,2012]:
  firms =  new_firms(full, dfs['df'+str(year)], 'NIT')
  full = full.merge(dfs['df'+str(year)], on='NIT', how='inner', suffixes=(None, 'to_remove'))
  duplicated  = [var for var in full.columns if 'to_remove' in var ]
  full.drop(columns=duplicated, inplace=True)
  full = pd.concat([full, firms])

In [28]:
full

,NIT,var1,var2,var3,year,indicator,cuatro
0,1,'A','B','C',2010,NaN,NaN
1,3,tres,'tres',10,2011,new,NaN
1,2,10,'12','12',2010,NaN,NaN
2,4,'cuatro','cuatro','cuatro',2012,new,yes


In [29]:
%%writefile clientes-month1.csv
cedula,feature1, feature2
1,  a,  10
2,  b,  12
3,  c,  20
5,  d,  13

Writing clientes-month1.csv


In [30]:
%%writefile clientes-month2.csv
cedula, feature1, feature2
1, a, 16
2, b, 17
3, c, 19
10, h, 0

Writing clientes-month2.csv


In [31]:
%%writefile clientes-month3.csv
cedula, feature1, feature2
1,  a,  0
3,  c,  1
11, j,  0

Writing clientes-month3.csv


In [32]:
to_load = ['clientes-month'+str(i) for i in range(1,4)]

In [33]:
df  = pd.read_csv('clientes-month3.csv')

In [34]:
df.rename(columns={'feature2':'he'})

,cedula,feature1,feature2
0,1,a,0
1,3,c,1
2,11,j,0


In [35]:
df.columns

Index(['cedula', ' feature1', ' feature2'], dtype='object')

In [36]:
df.columns = [var.replace(" ","") for var in df.columns]

In [37]:
df.columns

Index(['cedula', 'feature1', 'feature2'], dtype='object')

In [38]:
dfs = {}
for i in range(1,4):
  dfs['month'+str(i)] = pd.read_csv('clientes-month'+str(i)+'.csv')
  dfs['month'+str(i)].columns = [var.replace(" ","") for var in dfs['month'+str(i)].columns]
  dfs['month'+str(i)].rename(columns={'feature2':'feature-2'+str(i)}, inplace=True)

In [39]:
dfs

{'month1':    cedula feature1  feature-21
 0       1        a          10
 1       2        b          12
 2       3        c          20
 3       5        d          13,
 'month2':    cedula feature1  feature-22
 0       1        a          16
 1       2        b          17
 2       3        c          19
 3      10        h           0,
 'month3':    cedula feature1  feature-23
 0       1        a           0
 1       3        c           1
 2      11        j           0}

In [40]:
## Merge the dataset
full = dfs['month1'].copy()

In [41]:
for i in range(2,4):
  clients= new_firms(full,dfs['month'+str(i)], 'cedula')
  full = full.merge(dfs['month'+str(i)], on='cedula', how='inner', suffixes=(None, 'to_remove'))
  duplicated = [x for x in full.columns if 'to_remove' in x]
  full.drop(columns=duplicated, inplace=True)
  full = pd.concat([full, clients])

In [42]:
full

,cedula,feature1,feature-21,feature-22,feature-23
0,1,a,10.0,16.0,0.0
1,3,c,20.0,19.0,1.0
1,2,b,12.0,17.0,NaN
3,5,d,13.0,NaN,NaN
3,10,h,NaN,0.0,NaN
2,11,j,NaN,NaN,0.0


In [43]:
## BONUS
## Select a variable according to the time of presence of a event...

In [44]:
%%writefile predict.csv
nit,var-month-1,var-month-2,var-month-3,var-month-4,event
1,nan,10,2,0.1,4
2,3,0.4,nan,nan,2
3,8.5,6.7,0.2,nan,3
4,4.7,nan,nan,nan,1
5,5.6,0.1,nan,nan,2
6,nan,nan,3.1,0.23,4
7,nan,2.3, 0.01,nan,3
8,1,2,1.8,0.234, 4

Writing predict.csv


In [45]:
df  = pd.read_csv('predict.csv')

In [46]:
df.columns

Index(['nit', 'var-month-1', 'var-month-2', 'var-month-3', 'var-month-4',
       'event'],
      dtype='object')

In [51]:
import numpy as np
lag = 3
time_base = 1
prefix = 'var-month-'
df['var*'] = np.nan
for firm in df.index:
  time_ocurrence = df.loc[firm, 'event']
  if time_ocurrence - lag >= time_base:
    df.loc[firm, 'var*'] = df.loc[firm, prefix + str(time_ocurrence-lag)]

In [52]:
df

,nit,var-month-1,var-month-2,var-month-3,var-month-4,event,var*
0,1,NaN,10.0,2.00,0.100,4,NaN
1,2,3.0,0.4,NaN,NaN,2,NaN
2,3,8.5,6.7,0.20,NaN,3,NaN
3,4,4.7,NaN,NaN,NaN,1,NaN
4,5,5.6,0.1,NaN,NaN,2,NaN
5,6,NaN,NaN,3.10,0.230,4,NaN
6,7,NaN,2.3,0.01,NaN,3,NaN
7,8,1.0,2.0,1.80,0.234,4,1.0


In [49]:
# A function
def varInEvent(time_base, prefix, var_time_event, df_, lag=1):
  df = df_.copy()
  df['var*'] = np.nan
  for firm in df.index:
    time_ocurrence = df.loc[firm, var_time_event]
    if time_ocurrence - lag >= time_base:
      df.loc[firm, 'var*'] = df.loc[firm, prefix + str(time_ocurrence-lag)]
  return df['var*']

In [50]:
varInEvent(time_base=1, prefix='var-month-', var_time_event='event', df_=df, lag=3)

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
5    NaN
6    NaN
7    1.0
Name: var*, dtype: float64